In [7]:
import csv

from bs4 import BeautifulSoup
import requests


In [5]:
base_url = "https://www.olx.uz"
url = 'https://www.olx.uz/oz/nedvizhimost/kvartiry/prodazha/tashkent/?currency=UYE'

In [1]:
def get_href(url_): # get hrefs of all houses from url  one by one
    page = requests.get(url_)
    soup = BeautifulSoup(page.content, 'html.parser')
    houses1 =[]


    # Find all divs with class 'something'
    tags = soup.find_all("a", class_="css-1tqlkj0")
    for a_tag in tags:
      houses1.append(base_url + a_tag["href"])
    return houses1

In [45]:
#get first page
houses = get_href(url)
#get all remaining pages
for i in range(2,25):
    houses.extend(get_href(url+'&page='+str(i)))
# remove duplicates
houses = list(dict.fromkeys(houses))
print(len(houses))

104


In [ ]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_house_data(house_url, i=None):
    try:
        page = requests.get(house_url, timeout=10)
        soup = BeautifulSoup(page.content, 'html.parser')


        tags = soup.find("div", class_="css-41yf00")
        if tags is None:
            return None  # Skip this URL
        # 1. scrape main house data
        p_tags = tags.find_all("p")
        tag_content = {}
        for tag in p_tags:
            l = tag.text.split(':')
            if len(l) > 1:
                tag_content[l[0].strip()] = l[1].strip()
        # 2.  find district location
        loc_tag = soup.find("p", class_="css-7wnksb")
        district = loc_tag.text.split(',')
        if len(district) > 1:
                tag_content['Tuman'] = district[1].split()[0].strip()
        # 3. Get price
        price_tag = soup.find("h3", class_="css-fqcbii")
        price = price_tag.text.split()
        tag_content['Narx'] = price[0]
        # end
        if i is not None:
            print(f"[{i}] Fetched: {house_url}")
        return tag_content

    except Exception as e:
        print(f"Error fetching {house_url}: {e}")
        return None

def get_data_parallel(house_url_list, max_workers=12):
    house_content = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_house_data, url, i): url for i, url in enumerate(house_url_list)}
        for future in as_completed(futures):
            result = future.result()
            if result:
                house_content.append(result)

    return house_content

# Example usage
data = get_data_parallel(houses, max_workers=12)


[6] Fetched: https://www.olx.uz/d/oz/obyavlenie/4-komnatnaya-kvartira-centr-goroda-c-2-novyj-lyuks-remont-pod-klyuch-ID42UyJ.html
[3] Fetched: https://www.olx.uz/d/oz/obyavlenie/novostrojka-3-komnatnaya-74m-tyazhelyj-lyuks-orientir-darxan-interhouse-ID44g4F.html
[2] Fetched: https://www.olx.uz/d/oz/obyavlenie/arzon-zhk-wiston-3kom-evro-74m2-darxan-interhouse-haqiqiy-ID43uKE.html
[4] Fetched: https://www.olx.uz/d/oz/obyavlenie/zhk-charx-novza-2-komnatnaya-korobka-52m2-samarkand-darvoza-ulica-ID44fRD.html
[0] Fetched: https://www.olx.uz/d/oz/obyavlenie/piramit-tower-40-etazh-70m2-whitebox-ul-bobura-i-ul-nukus-ID44fZB.html
[1] Fetched: https://www.olx.uz/d/oz/obyavlenie/vtorichka-1-4-5-orientir-massiv-kushbegi-39m2-srochno-ID44g1O.html
[10] Fetched: https://www.olx.uz/d/oz/obyavlenie/novostrojku-3-2-9-parkentskij-65m2-evroremont-super-cena-ID41GLf.html
[9] Fetched: https://www.olx.uz/d/oz/obyavlenie/zhk-metropol-3-komnatnaya-87m2-biznes-klassa-novostrojka-yaponskij-sad-or-ID41AYC.html
[11

In [4]:
features = ['Turarjoy turi','Xonalar soni','Umumiy maydon','Qavati', 'Uy qavatliligi','Mebelli','Tuman','Narx']

In [134]:
import csv
# save data to csv file
with open('housing_data.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=features)
    writer.writeheader()
    for row in data:
        writer.writerow({k: row.get(k, '') for k in features})


data analysis

In [15]:
import csv
# read data to csv file
with open('housing_data.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f) # it gets headers and makes dictionaries  for every row
    data = [row for row in reader]
data

[{'Turarjoy turi': 'Ikkilamchi bozor',
  'Xonalar soni': '4',
  'Umumiy maydon': '95',
  'Qavati': '2',
  'Uy qavatliligi': '9',
  'Mebelli': 'Ha',
  'Tuman': 'Yunusobod',
  'Narx': '165'},
 {'Turarjoy turi': 'Yangi qurilgan uylar',
  'Xonalar soni': '3',
  'Umumiy maydon': '76',
  'Qavati': '8',
  'Uy qavatliligi': '10',
  'Mebelli': 'Ha',
  'Tuman': 'Mirzo-Ulug‘bek',
  'Narx': '185'},
 {'Turarjoy turi': 'Yangi qurilgan uylar',
  'Xonalar soni': '3',
  'Umumiy maydon': '74',
  'Qavati': '8',
  'Uy qavatliligi': '11',
  'Mebelli': 'Ha',
  'Tuman': 'Mirzo-Ulug‘bek',
  'Narx': '184'},
 {'Turarjoy turi': 'Yangi qurilgan uylar',
  'Xonalar soni': '2',
  'Umumiy maydon': '52',
  'Qavati': '11',
  'Uy qavatliligi': '12',
  'Mebelli': 'Yoʻq',
  'Tuman': 'Shayxontohur',
  'Narx': '56'},
 {'Turarjoy turi': 'Yangi qurilgan uylar',
  'Xonalar soni': '2',
  'Umumiy maydon': '70',
  'Qavati': '40',
  'Uy qavatliligi': '48',
  'Mebelli': 'Yoʻq',
  'Tuman': 'Yakkasaroy',
  'Narx': '170'},
 {'Turarjoy

In [9]:
# distinct districts
districts = set ([data[i]['Tuman'] for i in range(len(data))])
dt ={}
for item in districts:
    dt[item]= 0
for i in range(len(data)):
    dt[data[i]['Tuman']] +=1
print(dt)
print(dt.keys())

NameError: name 'data' is not defined